In [2]:
import sys
from pathlib import Path

In [3]:
project_directory = str(Path('../..').resolve())
sys.path.extend([project_directory])

In [12]:
from typing import Union
from os import getcwd, path

In [13]:
from PIL import Image
import pandas as pd
import numpy as np

In [14]:
from src.features.base.extract_process.extract_process import ExtractProcess
from src.features.mnist.pkg.image.image_to_nd_array import image_to_nd_array
from src.features.mnist.pkg.image.nd_array_to_df import img_nd_array_to_df
from src.features.mnist.pkg.clusters.center_of_mass import calculate_center_of_mass
from src.features.mnist.pkg.clusters.k_mean_clusters import calculate_k_means_clusters
from src.features.mnist.pkg.clusters.relative_coordinates import transform_to_relative_coordinates
from src.features.mnist.pkg.constants.df_scale_key import _construct_df_scale_key

In [15]:
cwd = getcwd()

In [17]:
x_key, y_key = 'x', 'y'
img_file_name = 'img_1.jpg'
scale = 16
cluster_size = 7

img_or_path = path.join(cwd, img_file_name)

In [21]:
image = image_to_nd_array(img_or_path, scale)
pd.DataFrame(image)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,124,119,0,0,0,0,0,0,0
3,0,0,0,0,0,199,255,255,255,244,149,0,0,0,0,0
4,0,0,0,0,173,255,255,218,197,255,255,192,0,0,0,0
5,0,0,0,0,254,204,140,0,0,0,253,255,112,0,0,0
6,0,0,0,154,255,0,0,0,0,0,120,255,198,0,0,0
7,0,0,0,182,246,0,0,0,0,0,0,238,237,0,0,0
8,0,0,0,246,253,0,0,0,0,0,0,165,255,0,0,0
9,0,0,0,222,255,0,0,0,0,0,0,189,246,0,0,0


In [23]:
center_of_mass = calculate_center_of_mass(image)
center_of_mass

(7.663063063063063, 7.589714714714715)

In [24]:
coordinates = img_nd_array_to_df(
    image,
    x_key, y_key,
    lambda each: each > 100
)
coordinates

,x,y
0,2,7
1,2,8
2,3,5
3,3,6
4,3,7
...,...,...
59,12,11
60,13,6
61,13,7
62,13,8


In [26]:
clusters = calculate_k_means_clusters(coordinates, cluster_size, x_key, y_key)
clusters

,x,y
0,4.125000,5.125000
1,12.090909,8.000000
2,5.500000,11.000000
3,10.875000,4.750000
4,9.800000,11.100000
5,3.111111,8.111111
6,7.500000,3.500000


In [28]:
relative_coordinates = transform_to_relative_coordinates(
    clusters,
    x_key, y_key,
    center_of_mass
)
relative_coordinates

,x,y,dx,dy,l,x_scale,y_scale
5,3.111111,8.111111,-4.551952,0.521396,4.581716,-1.000000,0.114543
1,12.090909,8.000000,4.427846,0.410285,4.446814,0.972736,0.090134
0,4.125000,5.125000,-3.538063,-2.464715,4.311926,-0.777263,-0.541463
3,10.875000,4.750000,3.211937,-2.839715,4.287251,0.705617,-0.623845
4,9.800000,11.100000,2.136937,3.510285,4.109574,0.469455,0.771160
6,7.500000,3.500000,-0.163063,-4.089715,4.092964,-0.035823,-0.898453
2,5.500000,11.000000,-2.163063,3.410285,4.038426,-0.475195,0.749192


In [30]:
a = relative_coordinates[_construct_df_scale_key(x_key)]
a

5   -1.000000
1    0.972736
0   -0.777263
3    0.705617
4    0.469455
6   -0.035823
2   -0.475195
Name: x_scale, dtype: float64

In [31]:
b = relative_coordinates[_construct_df_scale_key(y_key)]
b

5    0.114543
1    0.090134
0   -0.541463
3   -0.623845
4    0.771160
6   -0.898453
2    0.749192
Name: y_scale, dtype: float64

In [32]:
features = pd.Series(np.column_stack((a, b)).flatten())
features

0    -1.000000
1     0.114543
2     0.972736
3     0.090134
4    -0.777263
5    -0.541463
6     0.705617
7    -0.623845
8     0.469455
9     0.771160
10   -0.035823
11   -0.898453
12   -0.475195
13    0.749192
dtype: float64